In [46]:

import tensorflow as tf
import keras

In [47]:

import os
train_path ='C:/Users/User/Desktop/Project ML/chest_xray/train'
val_path = 'C:/Users/User/Desktop/Project ML/chest_xray/val'
test_path = 'C:/Users/User/Desktop/Project ML/chest_xray/test'

In [48]:
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Softmax,Input,Flatten,BatchNormalization

In [49]:
model1 = keras.models.Sequential()

model1.add(keras.layers.Flatten(input_shape=(150,150,1)))
model1.add(keras.layers.BatchNormalization())
model1.add(keras.layers.Dense(1024, activation='relu'))
model1.add(keras.layers.Dropout(0.5))
model1.add(keras.layers.Dense(512, activation='relu'))
model1.add(keras.layers.BatchNormalization())
model1.add(keras.layers.Dense(256, activation='relu'))
model1.add(keras.layers.Dense(128, activation='relu'))
model1.add(keras.layers.Dense(2, activation='softmax'))


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [50]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.98):
            print("\nReached 98% accuracy so cancelling training!")
            self.model.stop_training = True

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rescale=1.0/255.0)
train_batches = gen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    color_mode="grayscale",
    shuffle=True,
    seed=1,
    batch_size=16
)

Found 1566 images belonging to 2 classes.


In [54]:
callback = myCallback()
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(train_batches, epochs=40, callbacks=[callback])


Epoch 1/40


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 272ms/step - accuracy: 0.8088 - loss: 0.4281
Epoch 2/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 272ms/step - accuracy: 0.9287 - loss: 0.1896
Epoch 3/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 271ms/step - accuracy: 0.9401 - loss: 0.1715
Epoch 4/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 267ms/step - accuracy: 0.9274 - loss: 0.1990
Epoch 5/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 271ms/step - accuracy: 0.9324 - loss: 0.1710
Epoch 6/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 271ms/step - accuracy: 0.9417 - loss: 0.1576
Epoch 7/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 270ms/step - accuracy: 0.9457 - loss: 0.1338
Epoch 8/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 264ms/step - accuracy: 0.9740 - loss: 0.0756
Epoch 9/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 257ms/step - accuracy: 0.9615 - loss: 0.1003
Epoch 10/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 259ms/step - accuracy: 0.9504 - loss: 0.1388
Epoch 11/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 256ms/step - accuracy: 0.9665 - loss: 0.0882
Epoch 12/40
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 262ms/ste

In [57]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_batches = gen.flow_from_directory(
    test_path,
    target_size=model1.input_shape[1:3],
    color_mode="grayscale",
    shuffle=False,
    seed=1,
    batch_size=8
)


p1 = model1.predict(test_batches, verbose=True)


pre1 = pd.DataFrame(p1)
pre1["filename"] = test_batches.filenames
pre1["label"] = pre1["filename"].str.contains("PNEUMONIA").astype(int)
pre1['pre'] = (pre1[1] > 0.5).astype(int)


Found 624 images belonging to 2 classes.
 1/78 ━━━━━━━━━━━━━━━━━━━━ 12s 161ms/step

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step


In [58]:
recall_score(pre1["label"],pre1["pre"])

0.9461538461538461

In [59]:
roc_auc_score(pre1["label"],pre1[1])

0.874890422967346

In [60]:
ans = []
for i in pre1[1]:
    if(i>0.9):
        ans.append(1)
    else:
        ans.append(0)
accuracy_score(pre1["label"], ans)  

0.8012820512820513

In [61]:
from sklearn.metrics import f1_score
f1_score(pre1['label'],ans)

0.8516746411483254

In [62]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(pre1["label"], ans)
confusion_matrix

array([[144,  90],
       [ 34, 356]], dtype=int64)

In [63]:
import numpy as np
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
print(FP,FN,TP)


[34 90] [90 34] [144 356]


In [67]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

true_labels = pre1['label']
predicted_labels = pre1['pre']


cm = confusion_matrix(true_labels, predicted_labels)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
f1 = f1_score(true_labels, predicted_labels)
auc_roc = roc_auc_score(true_labels, pre1[1])  
precision = cm[0, 0] / (cm[0, 0] + cm[0, 1])

# Print metrics
print("MLP Model Metrics: ")
print("Accuracy: ", accuracy)
print("Sensitivity: ", sensitivity)
print("Specificity: ", specificity)
print("F1 Score: ", f1)
print("AUC-ROC: ", auc_roc)
print("Precision: ", precision)


MLP Model Metrics: 
Accuracy:  0.782051282051282
Sensitivity:  0.5085470085470085
Specificity:  0.9461538461538461
F1 Score:  0.8443935926773455
AUC-ROC:  0.874890422967346
Precision:  0.5085470085470085


In [68]:
model1.save('MLP.h5')

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

new_model = tf.keras.models.load_model('MLP.h5')

new_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 22500)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 22500)          │        90,000 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    23,041,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,822,356 (90.88 MB)

 Trainable params: 23,776,330 (90.70 MB)

 Non-trainable params: 46,024 (179.78 KB)

 Optimizer params: 2 (12.00 B)

In [10]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model 


train_path = 'C:/Users/User/Desktop/Project ML/chest_xray/train'
test_path = 'C:/Users/User/Desktop/Project ML/chest_xray/test'


gen = ImageDataGenerator(rescale=1.0/255.0)


train_batches = gen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    color_mode="grayscale",
    shuffle=True,
    seed=1,
    batch_size=16
)


test_batches = gen.flow_from_directory(
    test_path,
    target_size=(150, 150),  
    color_mode="grayscale",
    shuffle=False,
    seed=1,
    batch_size=8
)


p1 = new_model.predict(test_batches, verbose=True)


pre1 = pd.DataFrame(p1)

pre1["filename"] = test_batches.filenames


pre1["label"] = pre1["filename"].str.contains("PNEUMONIA").astype(int)


pre1['pre'] = (pre1[1] > 0.5).astype(int)


print(pre1.head())


Found 1566 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
 2/78 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step 

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
          0         1                  filename  label  pre
0  0.601076  0.398924  NORMAL\IM-0001-0001.jpeg      0    0
1  0.012698  0.987302  NORMAL\IM-0003-0001.jpeg      0    1
2  0.056101  0.943899  NORMAL\IM-0005-0001.jpeg      0    1
3  0.867424  0.132576  NORMAL\IM-0006-0001.jpeg      0    0
4  0.413268  0.586732  NORMAL\IM-0007-0001.jpeg      0    1


In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score
# Generate predictions and true labels
true_labels = pre1['label']
predicted_labels = pre1['pre']

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
f1 = f1_score(true_labels, predicted_labels)
auc_roc = roc_auc_score(true_labels, pre1[1])  # Assuming pre1[1] contains probability scores
precision = cm[0, 0] / (cm[0, 0] + cm[0, 1])

# Print metrics
print("MLP Model Metrics: ")
print("Accuracy: ", accuracy)
print("Sensitivity: ", sensitivity)
print("Specificity: ", specificity)
print("F1 Score: ", f1)
print("AUC-ROC: ", auc_roc)
print("Precision: ", precision)

MLP Model Metrics: 
Accuracy:  0.782051282051282
Sensitivity:  0.5085470085470085
Specificity:  0.9461538461538461
F1 Score:  0.8443935926773455
AUC-ROC:  0.874890422967346
Precision:  0.5085470085470085
